# Tarea 06

## Entrega: Miércoles 7 de octubre de 2015

[1] [Puedes brincar directo a la parte (iv).]

(i) Escribe una versión de Euler que funcione para un número arbitrario de variables. Acepta una función $\mathbf{f}: \mathbb{R}^n \to \mathbb{R}^n$, es decir, una función que come un vector y escupe otro. La estructura debe consistir en una función que actualiza las variables en un solo paso, y otra que integra desde un tiempo inicial hasta un tiempo final y regresa la trayectoria.

(ii) Escribe una versión de Runge-Kutta de segundo orden que funcione para un número arbitrario de variables, con la misma estructura.

(iii) Escribe una versión de Runge-Kutta de cuarto orden que funcione para un número arbitrario de variables, con la misma estructura.

(iv) Observando la similitud alta de las funciones que has hecho, has una función `integrar`, en el cual especifiques un parámetro adicional, `metodo`, que escoge el método (Euler, RK2 o RK4) para cada paso.

In [3]:
### Solución
# La función f aceptará argumentos de la forma {x, t}, donde x es un vector arbitrario, y regresará un vector de la 
# misma dimensión tal que x'(t) = f(x(t), t).

# Además, cualquiera de las funciones individuales relacionadas a los métodos de integración aceptará una celda
# de la forma {x_n, t_n}, donde x_n es un vector de por sí. Después, lo modificará en su lugar para no tener que pasar
# valores.

function euler_update!(values, f, h)
    values[1] += f(values) # Modificamos el vector
    values[2] += h # Movemos el tiempo
end;

function rk2_update!(values, f, h)
    k1 = f(values)
    k2 = f({values[1] + h * k1/2, values[2] + h/2})
    
    # Actualizamos el vector de estado
    values[1] += h*k2;
    values[2] += h;
end;

function rk4_update!(values, f, h)
    k1 = f(values)
    k2 = f({values[1] + h * k1 / 2, values[2] + h/2})
    k3 = f({values[1] + h * k2 / 2, values[2] + h/2})
    k4 = f({values[1] + h * k3, values[2] + h})
    
    # Actualizamos el vector de estado
    values[1] += h/6 * (k1 + 2*k2 + 2*k3 + k4);
    values[2] += h;
end;

# La función acepta las condiciones iniciales, el incremento de tiempo, el tiempo final,
# la función de evolución y el método a usar. Regresa el estado final.
function integrar(cond_iniciales, h, t_final, f, metodo=rk4_update!)
    estado = cond_iniciales # Inicializamos el estado
    
    while estado[2] < t_final # Iteramos hasta llegar al final
        metodo(estado, f, h);
    end
    
    return estado;
end

integrar (generic function with 2 methods)

[2] 

(i) Considera el tiro parabólico (sin fricción). Escribe con cuidado las ecuaciones diferenciales, identificando las variables. Escribe la función $\mathbf{f}$ correspondiente en Julia, que acepta los valores de las variables y regresa las derivadas corresopndientes.

(ii) Utiliza tus funciones para resolver numéricamente las ecuaciones del tiro parabólico (sin fricción) y compara con la solución analítica, con los tres métodos.

(iii) ¿Cómo varía la energía total en el tiempo con cada uno de los métodos? Según la dinámica clásica, ¿cómo debe variar? ¿Qué concluyes?

[3] 

(i) Agrega fricción proporcional a la *velocidad*. Identifica las variables necesarias y escribe las EDOs.

(ii) Escribe una versión diferente de Runge-Kutta que acepta otra función, que a su vez acepta el valores actual del vector de las variables. Esta función devolverá un valor booleano, que indica si cierta condición se satisface o no en base al vector de variables.

(iii) Escribe una función que dice si la posición actual haya llegado de regreso a la tierra. Empezando desde alguna altura sobre la tierra, corre el tiro no-parabólico (es decir, con fricción) *hasta que* llegue de regreso a la tierra.

(iv) Haz interactivo tu simulación, para poder cambiar los distintos parámetros.

(v) Dibuja el tiempo de vuelo y el alcance de la partícula en función del ángulo inicial para distintos tamaños de fricción.

(vi) ¿Cómo cambian los resultados si la fricción es proporcional al cuadrado de la *rapidez*?
[Pista: la fuerza de fricción siempre debe ser un *vector*. ¿En qué dirección apunta?] 